<a href="https://colab.research.google.com/github/vasudev-sharma/Flixstock-assignment/blob/master/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# PyTorch libraries
import torch
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import os
from pathlib import Path
import pandas as pd
from PIL import Image
import numpy as np
import math
import random
from tqdm import tqdm
import shutil

In [ ]:
!pip install wandb
# Visulazing the training logs, metrics and artifacts
import wandb



# Preparing the data

In [ ]:
# Download the data
!FILEID='1p7-dU6rDuqZ2mxv5ac5AWndt4z19aS6j' && \
FILENAME='data.zip' && \
FILEDEST="https://docs.google.com/uc?export=download&id=${FILEID}" && \
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate ${FILEDEST} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=${FILEID}" -O $FILENAME && rm -rf /tmp/cookies.txt

In [ ]:
# unzip the data
%%bash
unzip data.zip -d data


In [ ]:
data_path = Path('/content/data/classification-assignment')

In [ ]:
# Remove `Thumbs.db` file from images directory if exsists

path_file = data_path / 'images' / 'Thumbs.db'
if os.path.exists(str(path_file)):
  print("The file exists")
  os.unlink(str(path_file))
  print('Thumbs.db file has been removed')

## Datasets and DataLoaders

In [ ]:
class FashionDataset(Dataset):
  def __init__(self, root, transform=None, target_transform=None):
    
    self.root = root
    self.transform = transform
    self.target_transform = target_transform
    self.samples = os.listdir(str(self.root / 'images'))

    # Attributes dataframe
    self.df_attributes = pd.read_csv(str(self.root / 'attributes.csv'))

    # Encode each attribute into one-hot encoding, and then concatenate the encoded labels to get targets
    self.preprocess_targets() 

  def __len__(self):
    return len(self.samples)
  
  def __getitem__(self, index):

    # shuffle the images list
    np.random.shuffle(self.samples)

    # name of the image
    filename_image = self.samples[index]

    try:
      image = Image.open(str(self.root / 'images' / filename_image)).convert('RGB')
    except Exception as e:
      print('Path of the image is', str(self.root / 'images' / filename_image))
      print('Unable to read the image')

    # retreive the specific row of given index
    df_row = self.df_attributes.loc[self.df_attributes['filename'] == filename_image]

    if self.transform is not None:
      image = self.transform(image)

    # Target
    target_start_idx = self.df_attributes.columns.get_loc('neck_0.0')
    target = torch.tensor(self.df_attributes.iloc[0].tolist()[target_start_idx:], dtype=torch.float32)

    # return target and labels
    return image, target
  
  def preprocess_targets(self):

    # Drop rows which have `NA`
    # TODO: Look at optimal ways of reducing bias
    self.df_attributes = self.df_attributes.dropna()

    # one hot encode the Neck attribute
    one_hot_neck = pd.get_dummies(self.df_attributes.neck, prefix='neck')

    # one hot encode the sleeve_length attribute
    one_hot_sleeve_length = pd.get_dummies(self.df_attributes.sleeve_length, prefix='sleeve_length')

    # one hot encode the pattern attribute
    one_hot_pattern = pd.get_dummies(self.df_attributes.pattern, prefix='pattern')

    # concatenate the one hot encoded attributes to dataframe
    self.df_attributes = pd.concat([self.df_attributes, one_hot_neck, one_hot_sleeve_length, one_hot_pattern], axis=1)

  def getImagesList(self):
    return self.samples


In [ ]:
# transforms 
train_transforms =  transforms.Compose(
    [
     transforms.Resize((256, 256)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
      )

valid_transforms = transforms.Compose(
          [transforms.Resize((256, 256)),
           transforms.RandomHorizontalFlip(),
           transforms.ToTensor(),
           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
          ]
          )


In [ ]:
# Dataset

dataset = FashionDataset(data_path, transform=train_transforms)
list_images = dataset.getImagesList()

In [ ]:
# split of dataset into train, valid, test datasets

train_split = 0.75
val_split = 0.15
test_split = 1 - (train_split + val_split)

len_train_dataset = math.ceil(train_split * len(dataset))
len_val_dataset = math.ceil(val_split * len(dataset))
len_test_dataset = len(dataset) - (len_train_dataset + len_val_dataset)


In [ ]:
ds_train, ds_valid, ds_test = random_split(dataset, [len_train_dataset, len_val_dataset, len_test_dataset])

In [ ]:
# sanity check 
assert len(ds_train) == len_train_dataset
assert len(ds_valid) == len_val_dataset
assert len(ds_test) == len_test_dataset

# Test directory

In [ ]:
# Test directory for testing the images
# Test directory containes all the images for testing the model

try:
  os.mkdir(str(data_path / 'test'))
except Exception as e:
  print('The directory already exists')

# Save test images in `test` directory
for index, image_name in tqdm(enumerate(list_images)):
  if index in ds_test.indices:
    shutil.copy(str(data_path / 'images' / image_name), str(data_path / 'test' / image_name))




In [ ]:
# zip the test data for downloading
%%bash
cd data/classification-assignment
zip -r test.zip test

# Note this test data has been uploaded to Gdrive

# Hyperparameters

In [ ]:
# hyperparameters
batch_size = 16
lr = 5e-5
epochs=30
log_freq = 10

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_labels = 21



In [ ]:
model_dict = {
    # DenseNet
    'densenet121': 1024,
    'densenet169': 1664,
    'densenet161': 2208,

    # ResNet
    'resnet50': 2048,
    'resnet101' : 2048,
    'resnet34': 512,

    # EfficientNet
    'efficientnet-b0':1280,
    'efficientnet-b3': 1536,
    'efficientnet-b5': 2048

}


model_name = 'efficientnet-b3'
# Change the model name depending which model you want to fine tune

In [ ]:
device

## Dataloaders

In [ ]:
# Dataloaders
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dl_valid = DataLoader(ds_valid, batch_size=batch_size, shuffle=True, pin_memory=True)
dl_test = DataLoader(ds_valid, batch_size=len(ds_test), shuffle=False, pin_memory=True)

In [ ]:
# sanity check the shape 
for batch in dl_train:
  img_batch, target_batch = batch
  print(img_batch.shape)
  print(target_batch.shape)
  break

In [ ]:
#@title
!wandb login 202040aaac395bbf5a4a47d433a5335b74b7fb0e

# Model Training

In [ ]:
# efficient net model -> https://github.com/lukemelas/EfficientNet-PyTorch
!pip install efficientnet_pytorch

In [ ]:
# load a pretrained Densenet 121 model for finetuing on the Chest X_ray images
from efficientnet_pytorch import EfficientNet

if not model_name.startswith('efficientnet'):
  model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=True)
else:
  model = EfficientNet.from_pretrained(model_name)


In [ ]:
# Densenet 
if model_name.startswith('densenet'): 
  model.classifier = nn.Linear(model_dict[model_name], n_labels)

if model_name.startswith('efficientnet'):
  print(model_dict[model_name])
  model._fc = nn.Linear(model_dict[model_name], n_labels)
else:
  model.fc = nn.Linear(model_dict[model_name], n_labels)

# Migrate the mode to device
model.to(device)

## Model Loss, Optmizer and Scheduler

In [ ]:
# 
model.loss_func = nn.BCELoss()
model.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model.optimizer, T_max=5)

In [ ]:
from sklearn.metrics import *

In [ ]:
def calculate_metrics(pred, target, threshold=0.5):
  pred = np.array(pred > threshold, dtype = float)
  return { 
      # Micro scores
      'micro/precision':precision_score(y_true=target, y_pred=pred, average='micro'),
      'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
      'micro/f1_score': f1_score(y_true=target, y_pred=pred, average='micro'),

      # Macro scores
      'macro/precision':precision_score(y_true=target, y_pred=pred, average='macro'),
      'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
      'macro/f1_score': f1_score(y_true=target, y_pred=pred, average='macro'),

  }

In [ ]:

def train_model(model, dl_train, dl_valid, epochs, log_freq):
  print('***************Training of the model has started**************')
  wandb.watch(model, log='all')

  iteration = 0
  for epoch in range(epochs+1):

    batch_losses = []
    preds_list = []
    targets_list = []
    for batch_idx, (features, targets) in enumerate(dl_train, 1):
      model.train()
      
      model.optimizer.zero_grad()

      # Migrate the features and targets to device
      features, targets = features.to(device), targets.to(device)

      # Forward pass
      logits = model(features)
      preds = torch.sigmoid(logits)

      # loss
      loss = model.loss_func(preds, targets)

      # metrics
      preds_list.extend(preds.cpu().detach().numpy())
      targets_list.extend(targets.cpu().detach().numpy())

      # backward pass
      loss.backward()

      # update weights
      model.optimizer.step()

      batch_losses.append(loss.item())
    
    loss_mean = np.mean(batch_losses)
    metrics_dict = calculate_metrics(np.array(preds_list), np.array(targets_list))
    
    print('[Epoch = % d]: train_loss = %.3f,  micro/precision = %.3f, micro/recall = %.3f, micro/f1_score = %.3f, macro/precision = %.3f, macro/recall = %.3f, macro/f1_score = %.3f' \
          % (epoch, loss_mean , metrics_dict['micro/precision'], metrics_dict['micro/recall'], metrics_dict['micro/f1_score'], metrics_dict['macro/precision'], metrics_dict['macro/recall'], metrics_dict['macro/f1_score']))

    print('====='*12)
    print('\n')

    # Validation 
    model.eval()
    with torch.no_grad():

      val_preds_list = []
      val_targets_list = []
      val_loss_sum = 0.0
      for val_batch_idx, (val_features, val_targets) in enumerate(dl_valid, 1):
          
          val_features, val_targets = val_features.to(device), val_targets.to(device)
          
          # Forward pass
          val_logits = model(val_features)
          val_preds = torch.sigmoid(val_logits)

          val_preds_list.extend(val_preds.cpu().numpy())
          val_targets_list.extend(val_targets.cpu().numpy())

          # val loss
          val_loss = model.loss_func(val_preds, val_targets).item()
          val_loss_sum+=val_loss

    val_metrics_dict = calculate_metrics(np.array(val_preds_list), np.array(val_targets_list))


    print('-> [Epoch = % d]: val_loss = %.3f,  val_micro/precision = %.3f, val_micro/recall = %.3f, val_micro/f1_score = %.3f, val_macro/precision = %.3f, val_macro/recall = %.3f, val_macro/f1_score = %.3f' \
          % (epoch, val_loss_sum / val_batch_idx, val_metrics_dict['micro/precision'], val_metrics_dict['micro/recall'], val_metrics_dict['micro/f1_score'], val_metrics_dict['macro/precision'], val_metrics_dict['macro/recall'], val_metrics_dict['macro/f1_score']))
    print('----'*12)
    print('\n')

    wandb.log({'epoch': epoch,
               'train_loss': loss_mean,
               'train_micro_precision': metrics_dict['micro/precision'],
                'train_micro_recall':metrics_dict['micro/recall'],
                'train_micro_f1_score': metrics_dict['micro/f1_score'],
                'train_macro_precision':metrics_dict['macro/precision'],
                'train_macro_recall': metrics_dict['macro/recall'],
                 'train_macro_f1_score': metrics_dict['macro/f1_score'],
                 'val_loss' : val_loss_sum / val_batch_idx,
                'val_micro_precision': val_metrics_dict['micro/precision'],
                'val_micro_recall': val_metrics_dict['micro/recall'],
                'val_micro_f1_score': val_metrics_dict['micro/f1_score'],
                'val_macro_precision': val_metrics_dict['macro/precision'],
                'val_macro_recall': val_metrics_dict['macro/recall'],
                'val_macro_f1_score': val_metrics_dict['macro/f1_score']

              })

  
  print('****************Model training completed******************')
  torch.save(model.state_dict(), model_name + '.h5')




In [ ]:
%%wandb

def main():
  # wandb initialize a new run
  wandb.init(project='Flixstock-assignment')
  wandb.watch_called = False

  config = wandb.config
  config.batch_size = batch_size
  config.epochs = epochs
  config.lr = lr
  config.seed = 42
  config.labels = n_labels
  config.device = device
  config.model_name = model_name
  config.log_freq = log_freq

  # set seed and set cuddn to deterministic for reproducible results
  torch.manual_seed(config.seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  np.random.seed(config.seed)
  random.seed(config.seed)

  # train the model
  train_model(model, dl_train, dl_valid, epochs, log_freq)

 
if __name__=='__main__':
  main()
  wandb.finish()